# Import de fonctions

In [8]:
import matplotlib.pyplot as plt
from sklearn.metrics import adjusted_rand_score as ARI
from sklearn.metrics import adjusted_mutual_info_score as AMI
from sklearn.metrics import balanced_accuracy_score
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import v_measure_score
from skimage.transform import resize
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from scipy.stats import wasserstein_distance
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import numpy as np
import datetime
import sys

from scipy.ndimage import grey_dilation, grey_erosion
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
import torchvision.models as models
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn as nn
import torch
import math


# Dataset des chiffres du MNIST
from keras.datasets import mnist

In [9]:
# Fonction pour afficher les données matricielles sous forme d'images
def display_image(X, y, n, label=False):
    dim = 32
    fig, axs = plt.subplots(n, n, figsize=(5,5))

    for i in range(n):
      for j in range(n):
        axs[i,j].imshow(X[j*n+i].reshape((dim,dim)), cmap='gray')
        axs[i,j].axis('off')
    plt.show()

In [10]:
class NTXentLoss(torch.nn.Module):
    def __init__(self, device, batch_size, temperature, use_cosine_similarity):
        super(NTXentLoss, self).__init__()
        self.batch_size = batch_size
        self.temperature = temperature
        self.device = device
        self.softmax = torch.nn.Softmax(dim=-1)
        self.mask_samples_from_same_repr = self._get_correlated_mask().type(torch.bool)
        self.similarity_function = self._get_similarity_function(use_cosine_similarity)
        self.criterion = torch.nn.CrossEntropyLoss(reduction="sum")

    def _get_similarity_function(self, use_cosine_similarity):
        if use_cosine_similarity:
            self._cosine_similarity = torch.nn.CosineSimilarity(dim=-1)
            return self._cosine_simililarity
        else:
            return self._dot_simililarity

    def _get_correlated_mask(self):
        diag = np.eye(2 * self.batch_size)
        l1 = np.eye((2 * self.batch_size), 2 * self.batch_size, k=-self.batch_size)
        l2 = np.eye((2 * self.batch_size), 2 * self.batch_size, k=self.batch_size)
        mask = torch.from_numpy((diag + l1 + l2))
        mask = (1 - mask).type(torch.bool)
        return mask.to(self.device)

    @staticmethod
    def _dot_simililarity(x, y):
        v = torch.tensordot(x.unsqueeze(1), y.T.unsqueeze(0), dims=2)
        # x shape: (N, 1, C)
        # y shape: (1, C, 2N)
        # v shape: (N, 2N)
        return v

    def _cosine_simililarity(self, x, y):
        # x shape: (N, 1, C)
        # y shape: (1, 2N, C)
        # v shape: (N, 2N)
        v = self._cosine_similarity(x.unsqueeze(1), y.unsqueeze(0))
        return v

    def forward(self, zis, zjs):
        representations = torch.cat([zjs, zis], dim=0)

        similarity_matrix = self.similarity_function(representations, representations)

        # filter out the scores from the positive samples
        l_pos = torch.diag(similarity_matrix, self.batch_size)
        r_pos = torch.diag(similarity_matrix, -self.batch_size)
        positives = torch.cat([l_pos, r_pos]).view(2 * self.batch_size, 1)

        negatives = similarity_matrix[self.mask_samples_from_same_repr].view(2 * self.batch_size, -1)

        logits = torch.cat((positives, negatives), dim=1)
        logits /= self.temperature

        labels = torch.zeros(2 * self.batch_size).to(self.device).long()
        loss = self.criterion(logits, labels)

        return loss / (2 * self.batch_size)

# Define models and datasets

In [11]:
pos_label = 7

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = np.array([np.pad(img, 2) for img in X_train])
X_test = np.array([np.pad(img, 2) for img in X_test])

X_train = X_train[y_train==7]
X_test = X_test[y_test==7]
y_train = y_train[y_train==7]
y_test = y_test[y_test==7]

# flatten the images
n_samples_train = len(X_train)
X_train = X_train.reshape((n_samples_train, -1))
X_train = X_train - np.min(X_train, 1)[:, None]
X_train = X_train / np.max(X_train, 1)[:, None]

n_samples_test = len(X_test)
X_test = X_test.reshape((n_samples_test, -1))
X_test = X_test - np.min(X_test, 1)[:, None]
X_test = X_test / np.max(X_test, 1)[:, None]

X_train = np.array(X_train).astype(np.float)
X_test = np.array(X_test).astype(np.float)
y_train = np.array(y_train).astype(np.float)
y_test = np.array(y_test).astype(np.float)

# load ground truth label
import json

label_mapping_seven = {'No bar':0, 'With bar':1}
with open("/content/drive/MyDrive/Neurospin/Thèse Robin/UCSL/mnist_labels.json") as json_file:
    ground_truth_dict = json.load(json_file)
    y_cluster_test = np.array([label_mapping_seven[ground_truth_dict[key]] for key in ground_truth_dict.keys()])

<ipython-input-11-d45a57ac09e1>:24: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X_train = np.array(X_train).astype(np.float)
<ipython-input-11-d45a57ac09e1>:25: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X_test = np.array(X_test).astype(np.float)
<ipython-input-11-d45a57ac09e1>:26: DeprecationWarning: `np.float` is a deprecated alias

In [12]:
class Erode(object):
    """Random erode transform"""
    def __init__(self, kernel):
        self.kernel = kernel
    def __call__(self, x):
        x = grey_erosion(x, size=self.kernel)
        return torch.tensor(x)

class Dilate(object):
    """Random dilate transform"""
    def __init__(self, kernel):
        self.kernel = kernel

    def __call__(self, x):
        x = grey_dilation(x, size=self.kernel)
        return torch.tensor(x)

class MyDataset(Dataset):
    def __init__(self, data, target, cluster_pseudo_labels=None, test=False):
        self.data = data
        self.target = torch.from_numpy(target).float()
        if cluster_pseudo_labels is not None :
          self.cluster_pseudo_labels = torch.from_numpy(cluster_pseudo_labels).float()
        else :
          self.cluster_pseudo_labels = None
        self.transform = transforms.Compose([
                            transforms.ToTensor(),
                            # transforms.RandomResizedCrop(32, scale=(0.75, 1)),  # MORPHO version of SimCLR
                            transforms.RandomRotation(25),
                            transforms.RandomApply([Dilate(np.random.randint(2, 4))], p=0.4),   # MORPHO version of SimCLR
                            transforms.RandomApply([Erode(np.random.randint(2, 4))], p=0.4),   # MORPHO version of SimCLR
                            transforms.RandomAffine(15, translate=(0.1, 0.1), shear=0.1),
                        ])
        self.test = test

    def __len__(self):
        return len(self.target)

    def __getitem__(self, index):
        x = self.data[index]
        x = np.reshape(x, (32, 32))
        if self.test :
          x_1 = torch.tensor(x)[None,:,:].float()
        else :
          x_1 = self.transform(x)
        x_2 = self.transform(x)
        y = self.target[index]
        if self.cluster_pseudo_labels is not None :
          y_pseudo_label = self.cluster_pseudo_labels[index]
          return x_1.cuda(), x_2.cuda(), y.cuda(), y_pseudo_label.cuda()
        return x_1.cuda(), x_2.cuda(), y.cuda()


In [13]:
# Simple
class SimpleConvNet(nn.Module):
    def __init__(self, nc, nz, n_clusters, kernel_size=7):
        super(SimpleConvNet, self).__init__()
        padding = kernel_size // 2
        layers = [
            nn.Conv2d(nc, 16, kernel_size=kernel_size, padding=padding),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True),
            nn.Conv2d(16, 32, kernel_size=kernel_size, padding=padding),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 64, kernel_size=kernel_size, padding=padding),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 128, kernel_size=kernel_size, padding=padding),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
        ]
        self.backbone = nn.Sequential(*layers)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

        self.projection_head = nn.Sequential(
                nn.Linear(128, hidden_mlp),
                nn.ReLU(inplace=True),
                nn.Linear(hidden_mlp, output_dim),
            )

    def forward(self, x):
        # convolutional backbone
        encoded_vectors = self.backbone(x)
        encoded_vectors = self.avgpool(encoded_vectors)
        encoded_vectors = torch.flatten(encoded_vectors, 1).view(encoded_vectors.size(0), -1)

        # non linear mlp head
        x = self.projection_head(encoded_vectors)
        x = nn.functional.normalize(x, dim=1, p=2)
        return x, encoded_vectors


# SimCLR

In [14]:
# number of epochs to train the model
n_epochs = 75
base_lr = 0.005
final_lr = 0.00005
wd = 1e-4
warmup_epochs = 10
start_warmup = 0

# model parameters
hidden_mlp = 128
output_dim = 128
nc = 1
n_clusters=2
batch_size = 512

for run in range(0, 3) :
  # build model
  encoder_model = SimpleConvNet(nc, hidden_mlp, output_dim, n_clusters).double().cuda()

  # transform features
  encoder_model.eval()
  X_test_latent = []
  for img in X_test :
    img = img.reshape((32, 32))
    img = torch.tensor(img).cuda().double()[None, None,:,:]
    _, encoded_vector = encoder_model.forward(img)
    X_test_latent.append(encoded_vector.cpu().detach().numpy()[0])
  X_test_latent = np.array(X_test_latent)
  # transform features
  X_train_latent = []
  for img in X_train :
    img = img.reshape((32, 32))
    img = torch.tensor(img).cuda().double()[None, None,:,:]
    _, encoded_vector = encoder_model.forward(img)
    X_train_latent.append(encoded_vector.cpu().detach().numpy()[0])
  X_train_latent = np.array(X_train_latent)
  # normalize
  scaler = RobustScaler().fit(X_train_latent)
  X_train_latent = scaler.transform(X_train_latent)
  X_test_latent = scaler.transform(X_test_latent)

  # KMeans training
  KMeans_method = KMeans(n_clusters=n_clusters).fit(X_train_latent)
  y_train_clustering_pseudo_labels = KMeans_method.predict(X_train_latent)
  y_test_clustering_pseudo_labels = KMeans_method.predict(X_test_latent)

  print("Cluster 0 number of samples : ", np.sum(y_test_clustering_pseudo_labels==0))
  print("Cluster 1 number of samples : ", np.sum(y_test_clustering_pseudo_labels==1))
  print("KM balanced accuracy score with hand-crafted labels : ",
              max(balanced_accuracy_score(y_cluster_test, y_test_clustering_pseudo_labels[:len(y_cluster_test)]),
              balanced_accuracy_score(1-y_cluster_test, y_test_clustering_pseudo_labels[:len(y_cluster_test)]))
              )
  print("KM ARI with hand-crafted labels : ", ARI(y_cluster_test, y_test_clustering_pseudo_labels[:len(y_cluster_test)]))
  print("KM AMI with hand-crafted labels : ", AMI(y_cluster_test, y_test_clustering_pseudo_labels[:len(y_cluster_test)]))

  initial_clustering_accuracies = max(balanced_accuracy_score(y_cluster_test, y_test_clustering_pseudo_labels[:len(y_cluster_test)]),
              balanced_accuracy_score(1-y_cluster_test, y_test_clustering_pseudo_labels[:len(y_cluster_test)]))
  print("Initial clustering accuracy : ", initial_clustering_accuracies)
  print('------------------------------------------------------------------')

  # dataloaders
  train_data = MyDataset(X_train, y_train)
  test_data = MyDataset(X_test, y_test)
  # prepare data loaders
  train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, drop_last=True)
  test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, drop_last=True)


  # specify loss function
  list_of_parameters = []
  list_of_parameters.extend(encoder_model.backbone.parameters())
  list_of_parameters.extend(encoder_model.projection_head.parameters())
  optimizer = torch.optim.Adam(list_of_parameters, lr=base_lr)
  warmup_lr_schedule = np.linspace(start_warmup, base_lr, len(train_loader) * warmup_epochs)
  iters = np.arange(len(train_loader) * (n_epochs+1 - warmup_epochs))
  cosine_lr_schedule = np.array([final_lr + 0.5 * (base_lr - final_lr) * (1 + \
                        math.cos(math.pi * t / (len(train_loader) * (n_epochs - warmup_epochs)))) for t in iters])
  lr_schedule = np.concatenate((warmup_lr_schedule, cosine_lr_schedule))
  criterion = NTXentLoss("cuda", batch_size, 0.1, True)

  train_losses = []
  test_losses = []
  test_accuracies = []
  train_accuracies = []
  clustering_stabilities = []
  clustering_accuracies = [initial_clustering_accuracies]

  # launch training
  for epoch in range(0, n_epochs+1):
      # monitor training loss
      train_loss = 0.0
      test_loss = 0.0

      ###################
      # train the model #
      ###################
      encoder_model.train()
      for it, data in enumerate(train_loader):
          # update learning rate
          iteration = epoch * len(train_loader) + it
          for param_group in optimizer.param_groups:
              param_group["lr"] = lr_schedule[iteration]
          # _ stands in for labels, here no need to flatten images
          images_i, images_j, labels = data
          # forward pass: compute predicted outputs by passing inputs to the model
          latent_vectors_i, _ = encoder_model.forward(images_i)
          latent_vectors_j, _ = encoder_model.forward(images_j)
          loss = criterion(latent_vectors_i, latent_vectors_j)
          # backward pass: compute gradient of the loss with respect to model parameters
          loss.backward()
          # perform a single optimization step (parameter update)
          optimizer.step()
          # update running training loss
          train_loss += loss.item()*images_i.size(0)
          # clear the gradients of all optimized variables
          optimizer.zero_grad()


      ###################
      # evaluate the model #
      ###################
      encoder_model.eval()
      with torch.no_grad():
        for data in test_loader:
          # _ stands in for labels, here no need to flatten images
          images_i, images_j, labels = data
          # clear the gradients of all optimized variables
          optimizer.zero_grad()
          # forward pass: compute predicted outputs by passing inputs to the model
          latent_vectors_i, _ = encoder_model.forward(images_i)
          latent_vectors_j, _ = encoder_model.forward(images_j)
          # calculate the loss
          test_loss_i = criterion(latent_vectors_i, latent_vectors_j)
          # update running testing loss
          test_loss += test_loss_i.item()*images_i.size(0)

      # print avg training statistics
      train_loss = train_loss/len(train_loader)
      test_loss = test_loss/len(test_loader)
      #
      train_losses.append(train_loss)
      test_losses.append(test_loss)
      print('Epoch: {} \tTraining Loss: {:.6f}'.format(
          epoch,
          train_loss
          ))
      print('         \tTesting Loss: {:.6f}'.format(
          test_loss
          ))

      # transform features
      encoder_model.eval()
      X_test_latent = []
      for img in X_test :
        img = img.reshape((32, 32))
        img = torch.tensor(img).cuda().double()[None, None,:,:]
        _, encoded_vector = encoder_model.forward(img)
        X_test_latent.append(encoded_vector.cpu().detach().numpy()[0])
      X_test_latent = np.array(X_test_latent)
      # transform features
      X_train_latent = []
      for img in X_train :
        img = img.reshape((32, 32))
        img = torch.tensor(img).cuda().double()[None, None,:,:]
        _, encoded_vector = encoder_model.forward(img)
        X_train_latent.append(encoded_vector.cpu().detach().numpy()[0])
      X_train_latent = np.array(X_train_latent)
      # normalize
      scaler = RobustScaler().fit(X_train_latent)
      X_train_latent = scaler.transform(X_train_latent)
      X_test_latent = scaler.transform(X_test_latent)

      # KMeans training
      KMeans_method = KMeans(n_clusters=n_clusters).fit(X_train_latent)
      y_train_clustering_pseudo_labels = KMeans_method.predict(X_train_latent)
      previous_y_test_clustering_pseudo_labels = y_test_clustering_pseudo_labels.copy()
      y_test_clustering_pseudo_labels = KMeans_method.predict(X_test_latent)

      clustering_stability = ARI(previous_y_test_clustering_pseudo_labels, y_test_clustering_pseudo_labels)
      clustering_stabilities.append(clustering_stability)

      print("Cluster 0 number of samples : ", np.sum(y_test_clustering_pseudo_labels==0))
      print("Cluster 1 number of samples : ", np.sum(y_test_clustering_pseudo_labels==1))
      print("KM balanced accuracy score with hand-crafted labels : ",
                  max(balanced_accuracy_score(y_cluster_test, y_test_clustering_pseudo_labels[:len(y_cluster_test)]),
                  balanced_accuracy_score(1-y_cluster_test, y_test_clustering_pseudo_labels[:len(y_cluster_test)]))
                  )
      clustering_accuracies.append( max(balanced_accuracy_score(y_cluster_test, y_test_clustering_pseudo_labels[:len(y_cluster_test)]),
                                        balanced_accuracy_score(1-y_cluster_test, y_test_clustering_pseudo_labels[:len(y_cluster_test)]))  )
      print("KM ARI with hand-crafted labels : ", ARI(y_cluster_test, y_test_clustering_pseudo_labels[:len(y_cluster_test)]))
      print("KM AMI with hand-crafted labels : ", AMI(y_cluster_test, y_test_clustering_pseudo_labels[:len(y_cluster_test)]))
      print('------------------------------------------------------------------')


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Cluster 0 number of samples :  584
Cluster 1 number of samples :  444
KM balanced accuracy score with hand-crafted labels :  0.5545090248998185
KM ARI with hand-crafted labels :  0.03977844186847393
KM AMI with hand-crafted labels :  0.006550287185749865
Initial clustering accuracy :  0.5545090248998185
------------------------------------------------------------------
Epoch: 0 	Training Loss: 3550.822264
         	Testing Loss: 3547.016145


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Cluster 0 number of samples :  423
Cluster 1 number of samples :  605
KM balanced accuracy score with hand-crafted labels :  0.5594924136041374
KM ARI with hand-crafted labels :  0.04583488890908152
KM AMI with hand-crafted labels :  0.008380321745474614
------------------------------------------------------------------
Epoch: 1 	Training Loss: 3535.516901
         	Testing Loss: 3510.428931


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


...

Cluster 0 number of samples :  464
Cluster 1 number of samples :  564
KM balanced accuracy score with hand-crafted labels :  0.722437236702401
KM ARI with hand-crafted labels :  0.2088985677804389
KM AMI with hand-crafted labels :  0.12677034690206102
------------------------------------------------------------------
Epoch: 74 	Training Loss: 1920.684217
         	Testing Loss: 2840.389361


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Cluster 0 number of samples :  466
Cluster 1 number of samples :  562
KM balanced accuracy score with hand-crafted labels :  0.7275918758776587
KM ARI with hand-crafted labels :  0.2144500928528572
KM AMI with hand-crafted labels :  0.1325488972845569
------------------------------------------------------------------
Epoch: 75 	Training Loss: 1912.260188
         	Testing Loss: 2838.233652


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Cluster 0 number of samples :  465
Cluster 1 number of samples :  563
KM balanced accuracy score with hand-crafted labels :  0.722437236702401
KM ARI with hand-crafted labels :  0.2088985677804389
KM AMI with hand-crafted labels :  0.12677034690206102
------------------------------------------------------------------
